In [1]:
import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from encoder import *
from AttnDecoder import * 
from seq2seq import *

from build_dataset import *

/ext3/conda/1011_project/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load data 
train_file_path = {
    'source': f"data/processed/src-train.txt",
    'target': f"data/processed/tgt-train.txt"
}

test_file_path = {
    'source': f"data/processed/src-test.txt",
    'target': f"data/processed/tgt-test.txt"
}

train_dataset = QAPair(train_file_path)
test_dataset = QAPair(test_file_path)

train_dataloader = DataLoader(
            train_dataset, 
            batch_size=128, 
            shuffle=True, 
            collate_fn=partial(pad_collate_fn, pad_token=train_dataset.pad_idx)
        )

test_dataloader = DataLoader(
            test_dataset, 
            batch_size=128, 
            shuffle=True, 
            collate_fn=partial(pad_collate_fn, pad_token=test_dataset.pad_idx)
        )

In [3]:
pretrained_vectors = {}
pretrained_vectors['enc'] = torch.load('embeddings/encoder_emb.pt')
pretrained_vectors['dec'] = torch.load('embeddings/decoder_emb.pt')

# input_size = len(train_dataset.answer_vocab)
embed_size = 50  
output_size = len(train_dataset.question_vocab) # is output size still vocab size of target? 

seq2seq = Seq2Seq(pretrained_vectors, output_size, embed_size)

In [4]:
def train_step(batch, model, optimizer, criterion, device):
    input, xs_len, target = batch.input_vecs.to(device), batch.input_lens.to(device), batch.target_vecs.to(device)

    model.enc.train()
    model.dec.train()

    s2s_output, s2s_hidden = model(input, xs_len)

    scores = s2s_output.view(-1, s2s_output.size(-1))
    
    optimizer.zero_grad()
    loss = criterion(scores, target.view(-1))
    loss.backward()
    optimizer.step()

    return loss.item()
    

In [5]:
from tqdm import tqdm 
# TRAIN LOOP 

seq2seq = Seq2Seq(pretrained_vectors, output_size, embed_size)
optim = torch.optim.SGD(seq2seq.parameters(), lr=0.1)
criterion = nn.NLLLoss()
device = "cuda" if torch.cuda.is_available() else "cpu"

seq2seq.to(device) 

for epoch in tqdm(range(1)):

    for i, data in tqdm(enumerate(train_dataloader)):
        curr_loss = train_step(data, seq2seq, optim, criterion, device)
        print(f'epoch {epoch+1}, loss = {curr_loss}')
# MOMENT OF TRUTH???
# ARE YALL READY 
# so the embedding part is correct ok that was my b i didn't know what input size was  

0it [00:00, ?it/s]/1 [00:00<?, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]


encoder_output shape in forward decoder torch.Size([128, 86, 1200])
encoder_output type in forward decoder torch.cuda.FloatTensor
encoder_output in forward decoder tensor([[[ 0.0020,  0.0207,  0.0013,  ...,  0.0067, -0.0546, -0.0065],
         [ 0.0073,  0.0423,  0.0201,  ...,  0.0188, -0.0526, -0.0072],
         [-0.0042,  0.0527,  0.0235,  ...,  0.0228, -0.0523, -0.0121],
         ...,
         [-0.0176,  0.0422,  0.0096,  ...,  0.0013, -0.0192, -0.0076],
         [-0.0183,  0.0413,  0.0092,  ...,  0.0021, -0.0160, -0.0061],
         [-0.0195,  0.0397,  0.0085,  ...,  0.0024, -0.0106, -0.0036]],

        [[ 0.0016,  0.0205,  0.0006,  ...,  0.0200, -0.0536, -0.0010],
         [ 0.0077,  0.0398,  0.0176,  ...,  0.0355, -0.0580,  0.0017],
         [ 0.0070,  0.0458,  0.0191,  ...,  0.0307, -0.0581,  0.0020],
         ...,
         [-0.0176,  0.0422,  0.0096,  ...,  0.0013, -0.0192, -0.0076],
         [-0.0183,  0.0413,  0.0092,  ...,  0.0021, -0.0160, -0.0061],
         [-0.0195,  0.039

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)